In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'ML/KRembedding/'

LIB_PATH = 'lib'
DATA_PATH = '/notebooks/embedding/data/'

%cd drive/My\ Drive/$FOLDERNAME/

# 형태소 분석 완료된 데이터 다운로드
!bash preprocess.sh dump-tokenized

Mounted at /content/drive
/content/drive/My Drive/ML/KRembedding
download tokenized data...
--2021-05-30 08:09:41--  https://docs.google.com/uc?export=download&confirm=ZfZZ&id=1Ybp_DmzNEpsBrUKZ1-NoPDzCMO39f-fx
Resolving docs.google.com (docs.google.com)... 142.250.65.78, 2607:f8b0:4004:832::200e
Connecting to docs.google.com (docs.google.com)|142.250.65.78|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0c-a8-docs.googleusercontent.com/docs/securesc/ncf9rmvblq6qmmatskv23vtprvo8ghro/i6a3oss0uenllpbcc6lq5s3i251qirtv/1622362125000/05170634686643261154/06696315714116104829Z/1Ybp_DmzNEpsBrUKZ1-NoPDzCMO39f-fx?e=download [following]
--2021-05-30 08:09:41--  https://doc-0c-a8-docs.googleusercontent.com/docs/securesc/ncf9rmvblq6qmmatskv23vtprvo8ghro/i6a3oss0uenllpbcc6lq5s3i251qirtv/1622362125000/05170634686643261154/06696315714116104829Z/1Ybp_DmzNEpsBrUKZ1-NoPDzCMO39f-fx?e=download
Resolving doc-0c-a8-docs.googleusercontent.com (doc-0c-a8-d

In [ ]:
# Mecab 설치
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Installing automake (A dependency for mecab-ko)
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [60.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/'+FOLDERNAME+LIB_PATH)

In [4]:
!cd $DATA_PATH/tokenized && cat wiki_ko_mecab.txt ratings_mecab.txt korquad_mecab.txt > corpus_mecab.txt

In [2]:
# GloVe 설치

%cd /content
!git clone https://github.com/stanfordnlp/glove
!cd glove && make
!bash demo.sh
%cd drive/My\ Drive/$FOLDERNAME/

/content
Cloning into 'glove'...
remote: Enumerating objects: 595, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 595 (delta 0), reused 1 (delta 0), pack-reused 592
Receiving objects: 100% (595/595), 222.33 KiB | 12.35 MiB/s, done.
Resolving deltas: 100% (338/338), done.
mkdir -p build
gcc -c src/vocab_count.c -o build/vocab_count.o -lm -pthread -O3 -march=native -funroll-loops -Wall -Wextra -Wpedantic
gcc -c src/cooccur.c -o build/cooccur.o -lm -pthread -O3 -march=native -funroll-loops -Wall -Wextra -Wpedantic
src/cooccur.c: In function ‘merge_files’:
src/cooccur.c:180:9: warning: ignoring return value of ‘fread’, declared with attribute warn_unused_result [-Wunused-result]
         fread(&new, sizeof(CREC), 1, fid[i]);
         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/cooccur.c:190:5: warning: ignoring return value of ‘fread’, declared with attribute warn_unused_result [-Wunused-result]
     fread(&new, sizeof(CREC), 1, 

In [6]:
%cd /content/glove/build
!mkdir -p $DATA_PATH/word-embeddings/glove
!./vocab_count -min-count 5 -verbose 2 \
  < $DATA_PATH/tokenized/corpus_mecab.txt > $DATA_PATH/word-embeddings/glove/glove.vocab
!./cooccur -memory 10.0 -verbose 2 -window-size 15 -vocab-file $DATA_PATH/word-embeddings/glove/glove.vocab \
  < $DATA_PATH/tokenized/corpus_mecab.txt > $DATA_PATH/word-embeddings/glove/glove.cooc
!./shuffle -memory 10.0 -verbose 2 \
  < $DATA_PATH/word-embeddings/glove/glove.cooc > $DATA_PATH/word-embeddings/glove/glove.shuf
!./glove -threads 4 -x-max 10 -iter 15 -vector-size 100 -binary 2 \
  -input-file $DATA_PATH/word-embeddings/glove/glove.shuf \
  -vocab-file $DATA_PATH/word-embeddings/glove/glove.vocab \
  -save-file $DATA_PATH/word-embeddings/glove/glove
%cd ../../drive/MyDrive/$FOLDERNAME

/content/glove/build
BUILDING VOCABULARY
Processed 0 tokens.100000 tokens.200000 tokens.300000 tokens.400000 tokens.500000 tokens.600000 tokens.700000 tokens.800000 tokens.900000 tokens.1000000 tokens.1100000 tokens.1200000 tokens.1300000 tokens.1400000 tokens.1500000 tokens.1600000 tokens.1700000 tokens.1800000 tokens.1900000 tokens.2000000 tokens.2100000 tokens.2200000 tokens.2300000 tokens.2400000 tokens.2500000 tokens.2600000 tokens.2700000 tokens.2800000 tokens.2900000 tokens.3000000 tokens.3100000 tokens.3200000 tokens.3300000 tokens.3400000 tokens.3500000 tokens.3600000 tokens.3700000 tokens.3800000 tokens.3900000 tokens.4000000 tokens.4100000 tokens.4200000 tokens.4300000 tokens.4400000 tokens.4500000 tokens.4600000 tokens.4700000 tokens.4800000 tokens.4900000 tokens.5000000 tokens.5100000 tokens.5200000 tokens.5300000 tokens.5400000 tokens.5500000 tokens.5600000 tokens.5700000 tokens.5800000 tokens.5900000 tokens.6000000 tokens.6100000 tokens.6200000 tokens.6300000 tokens.6400